In [1]:
import torch, pickle, yaml
all_smiles_train_2d = pickle.load(open('/workspace/SMILES_dataset/train/SMILES/index.pkl', 'rb'))
all_smiles_val_2d = pickle.load(open('/workspace/SMILES_dataset/val/SMILES/index.pkl', 'rb'))
all_smiles_test_2d = pickle.load(open('/workspace/SMILES_dataset/test/SMILES/index.pkl', 'rb'))

all_chemical_names_train_2d = pickle.load(open('/workspace/SMILES_dataset/train/Chemical/index.pkl', 'rb'))
all_chemical_names_val_2d = pickle.load(open('/workspace/SMILES_dataset/val/Chemical/index.pkl', 'rb'))
all_chemical_names_test_2d = pickle.load(open('/workspace/SMILES_dataset/test/Chemical/index.pkl', 'rb'))

all_smiles_train_1d = pickle.load(open('/workspace/OneD_Only_Dataset/train/SMILES/index.pkl', 'rb'))
all_smiles_val_1d = pickle.load(open('/workspace/OneD_Only_Dataset/val/SMILES/index.pkl', 'rb'))
all_smiles_test_1d = pickle.load(open('/workspace/OneD_Only_Dataset/test/SMILES/index.pkl', 'rb'))

all_chemical_names_train_1d = pickle.load(open('/workspace/OneD_Only_Dataset/train/Chemical/index.pkl', 'rb'))
all_chemical_names_val_1d = pickle.load(open('/workspace/OneD_Only_Dataset/val/Chemical/index.pkl', 'rb'))
all_chemical_names_test_1d = pickle.load(open('/workspace/OneD_Only_Dataset/test/Chemical/index.pkl', 'rb'))

import imp
smart = imp.load_source('/root/MorganFP_prediction/reproduce_previous_works/smart4.5/datasets/', '/root/MorganFP_prediction/reproduce_previous_works/smart4.5/datasets/dataset_utils.py')
    


/tmp/ipykernel_45561/1605590043.py:18: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp


In [8]:
all_smiles_val_1d

{30: 'O=C(O)C#Cc1ccccc1',
 36: 'O=c1ccn([C@@H]2O[C@H](CO[P@@](=O)(O)O[P@@](=O)(O)O[C@H]3O[C@H](CO)[C@H](O)[C@H](O)[C@H]3O)[C@@H](O)[C@H]2O)c(=O)[nH]1',
 39: 'N[C@H]1C(O)O[C@H](CO)[C@@H](O)[C@@H]1O',
 61: 'CCCCC[C@H](O)/C=C/[C@H]1C(=O)C[C@H](O)[C@@H]1C/C=C\\CCCC(=O)O',
 87: 'O=C(O)[C@H](O)COP(=O)(O)O',
 91: 'O=C[C@H](O)[C@@H](O)[C@@H](O)[C@H](O)C(=O)O',
 95: 'CCCCC[C@H](O)/C=C/[C@H]1[C@H](O)CC(=O)[C@@H]1C/C=C\\CCCC(=O)O',
 98: 'O=c1ccn([C@@H]2O[C@H](COP(=O)(O)OP(=O)(O)O[C@H]3O[C@H](CO)[C@@H](O)[C@H](O)[C@H]3O)[C@@H](O)[C@H]2O)c(=O)[nH]1',
 117: 'CCCCC/C=C\\C/C=C\\C/C=C\\CCCCCCC(=O)O',
 121: 'O=P(O)(O)OC[C@H]1O[C@@](CO)(OP(=O)(O)O)[C@@H](O)[C@@H]1O',
 123: 'CC1(C)C2CC[C@]3(C)C(C(=O)C=C4C5C[C@@](C)(C(=O)O)CC[C@]5(C)CC[C@]43C)[C@@]2(C)CC[C@@H]1O',
 125: 'NC(=O)C1=CN([C@H]2O[C@@H](COP(=O)(O)OP(=O)(O)OC[C@@H]3O[C@H](n4cnc5c(N)ncnc54)[C@@H](OP(=O)(O)O)[C@H]3O)[C@H](O)[C@@H]2O)C=CC1',
 134: 'NC(=O)NCCCC[C@H](N)C(=O)O',
 146: 'CCCCCCCCCCCCCCCC(=O)OC[C@H](COP(=O)(O)O)OC(=O)CCCCCCCCCCCCCCC',
 152

In [3]:
import sys, os 
from rdkit import Chem

def isomeric_to_canonical_smiles(isomeric_smiles):

    mol = Chem.MolFromSmiles(isomeric_smiles)
    Chem.RemoveStereochemistry( mol ) 

    canonical_smiles = Chem.MolToSmiles(mol, canonical=True)
    
    return canonical_smiles

In [6]:
import numpy as np
from rdkit import Chem

from rdkit.Chem import rdMolDescriptors
def generate_normal_FP(mol, radius=2, length=6144):

    # Dictionary to store information about which substructures contribute to setting which bits
    bitInfo = {}
    mol_H = Chem.AddHs(mol) # add implicit Hs to the molecule

    
    # Generate the fingerprint with bitInfo to track the substructures contributing to each bit
    fp = rdMolDescriptors.GetMorganFingerprintAsBitVect(mol_H, radius=radius, nBits=length, bitInfo=bitInfo)
    new_fp = [0] * length
    for bit, atoms in bitInfo.items():
            new_fp[bit] = 1
            
    return np.array(new_fp)

In [12]:
for idx, iso_smiles in all_smiles_train_2d.items():
    can_smiles = isomeric_to_canonical_smiles(iso_smiles)
    iso_fp = generate_normal_FP(Chem.MolFromSmiles(iso_smiles))
    can_fp =  generate_normal_FP(Chem.MolFromSmiles(can_smiles))
    if (iso_fp==can_fp).all() == False:
        print("different!")

[14:25:02] WARNING: not removing hydrogen atom without neighbors
[14:25:02] WARNING: not removing hydrogen atom without neighbors
[14:25:02] WARNING: not removing hydrogen atom without neighbors
[14:25:15] WARNING: not removing hydrogen atom without neighbors
[14:25:15] WARNING: not removing hydrogen atom without neighbors
[14:25:15] WARNING: not removing hydrogen atom without neighbors
[14:25:15] WARNING: not removing hydrogen atom without neighbors
[14:25:15] WARNING: not removing hydrogen atom without neighbors
[14:25:15] WARNING: not removing hydrogen atom without neighbors
[14:25:54] WARNING: not removing hydrogen atom without neighbors
[14:25:54] WARNING: not removing hydrogen atom without neighbors
[14:25:54] WARNING: not removing hydrogen atom without neighbors
[14:25:54] WARNING: not removing hydrogen atom without neighbors
[14:25:54] WARNING: not removing hydrogen atom without neighbors
[14:25:54] WARNING: not removing hydrogen atom without neighbors
[14:26:38] WARNING: not r

In [11]:
iso_fp.nonzero(), can_fp.nonzero()

((array([  49,  363,  364,  685,  896,  945, 1064, 1073, 1117, 1279, 1341,
         1365, 1380, 1395, 1607, 1757, 1905, 2409, 2512, 2623, 2650, 2698,
         2704, 2714, 2793, 2798, 2867, 2965, 2974, 3067, 3068, 3087, 3183,
         3286, 3383, 3385, 3417, 3447, 3748, 3821, 3858, 4283, 4398, 4410,
         4448, 4462, 4647, 4744, 4903, 4995, 5008, 5046, 5104, 5117, 5156,
         5175, 5390, 5418, 5632, 5748, 5969]),),
 (array([  49,  363,  364,  685,  896,  945, 1064, 1073, 1117, 1279, 1341,
         1365, 1380, 1395, 1607, 1757, 1905, 2409, 2512, 2623, 2650, 2698,
         2704, 2714, 2793, 2798, 2867, 2965, 2974, 3067, 3068, 3087, 3183,
         3286, 3383, 3385, 3417, 3447, 3748, 3821, 3858, 4283, 4398, 4410,
         4448, 4462, 4647, 4744, 4903, 4995, 5008, 5046, 5104, 5117, 5156,
         5175, 5390, 5418, 5632, 5748, 5969]),))

In [11]:
import tqdm
for split in ['train', 'val', 'test']:
        valid_indices = []
        all_smiles = locals()['all_smiles_' + split + '_1d' ]
        dataset_dir =  '/workspace/OneD_Only_Dataset/'
        out_path = dataset_dir + split + '/SMILES/index.pkl'
        print(out_path)
        os.system(f"mkdir {dataset_dir + split + '/Isomeric_SMILES/'}")
        os.system(f"cp {out_path} {out_path.replace('SMILES', 'Isomeric_SMILES')}")
        for k,v in tqdm.tqdm(all_smiles.items()):
            all_smiles[k] = isomeric_to_canonical_smiles(v)
        pickle.dump(all_smiles, open(out_path, 'wb'))

/workspace/OneD_Only_Dataset/train/SMILES/index.pkl


  0%|          | 0/177662 [00:00<?, ?it/s]

100%|██████████| 177662/177662 [00:39<00:00, 4491.23it/s]


/workspace/OneD_Only_Dataset/val/SMILES/index.pkl


100%|██████████| 22157/22157 [00:04<00:00, 4469.03it/s]
mkdir: cannot create directory ‘/workspace/OneD_Only_Dataset/test/Isomeric_SMILES/’: File exists


/workspace/OneD_Only_Dataset/test/SMILES/index.pkl


100%|██████████| 22001/22001 [00:04<00:00, 4715.72it/s]


In [15]:
# mol = Chem.MolFromSmiles("NC(=O)c1ccc[n+](C2OC(COP(=O)(O)OP(=O)(O)OCC3OC(n4cnc5c(N)ncnc54)C(O)C3O)C(O)C2O)c1")
# img = Draw.MolToImage(mol)
# img.show()

In [18]:
all_smiles_train_1d = pickle.load(open('/workspace/OneD_Only_Dataset/train/SMILES/index.pkl', 'rb'))
all_smiles_val_1d = pickle.load(open('/workspace/OneD_Only_Dataset/val/SMILES/index.pkl', 'rb'))
all_smiles_test_1d = pickle.load(open('/workspace/OneD_Only_Dataset/test/SMILES/index.pkl', 'rb'))

all_chemical_names_train_1d = pickle.load(open('/workspace/OneD_Only_Dataset/train/Chemical/index.pkl', 'rb'))
all_chemical_names_val_1d = pickle.load(open('/workspace/OneD_Only_Dataset/val/Chemical/index.pkl', 'rb'))
all_chemical_names_test_1d = pickle.load(open('/workspace/OneD_Only_Dataset/test/Chemical/index.pkl', 'rb'))


In [25]:
# merge_all_2d_smiles
all_smiles_1d=[]
for split in ['train', 'val', 'test']:
        valid_indices = []
        all_smiles = locals()['all_smiles_' + split + '_1d' ]
        dataset_dir =  f'/workspace/OneD_Only_Dataset/{split}/oneD_NMR/'
        for file_id in os.listdir(dataset_dir):
            int_id = int(file_id.split('.')[0])
            all_smiles_1d.append(all_smiles[int_id])

In [26]:
len(all_smiles_1d), len(set(all_smiles_1d))

(144001, 124505)

In [27]:
all_smiles_2d=[]
for split in ['train', 'val', 'test']:
        valid_indices = []
        all_smiles = locals()['all_smiles_' + split + '_2d' ]
        dataset_dir =  f'/workspace/SMILES_dataset/{split}/HSQC/'
        for file_id in os.listdir(dataset_dir):
            int_id = int(file_id.split('.')[0])
            all_smiles_2d.append(isomeric_to_canonical_smiles(all_smiles[int_id]))

[23:48:21] WARNING: not removing hydrogen atom without neighbors
[23:48:22] WARNING: not removing hydrogen atom without neighbors
[23:48:25] WARNING: not removing hydrogen atom without neighbors
[23:48:29] WARNING: not removing hydrogen atom without neighbors
[23:48:33] WARNING: not removing hydrogen atom without neighbors
[23:48:33] WARNING: not removing hydrogen atom without neighbors
[23:48:34] WARNING: not removing hydrogen atom without neighbors
[23:48:36] WARNING: not removing hydrogen atom without neighbors
[23:48:37] WARNING: not removing hydrogen atom without neighbors
[23:48:37] WARNING: not removing hydrogen atom without neighbors
[23:48:44] WARNING: not removing hydrogen atom without neighbors
[23:48:45] WARNING: not removing hydrogen atom without neighbors
[23:48:45] WARNING: not removing hydrogen atom without neighbors
[23:48:47] WARNING: not removing hydrogen atom without neighbors
[23:48:49] WARNING: not removing hydrogen atom without neighbors


In [28]:
len(all_smiles_2d), len(set(all_smiles_2d))

(137062, 111355)

In [29]:
all_smiles_2d=[]
for split in ['train', 'val', 'test']:
        valid_indices = []
        all_smiles = locals()['all_smiles_' + split + '_2d' ]
        dataset_dir =  f'/workspace/SMILES_dataset/{split}/HSQC/'
        for file_id in os.listdir(dataset_dir):
            int_id = int(file_id.split('.')[0])
            all_smiles_2d.append((all_smiles[int_id]))

In [30]:
len(all_smiles_2d), len(set(all_smiles_2d))

(137062, 111361)

In [31]:
overlap_2d1d = set(all_smiles_2d).intersection(set(all_smiles_1d))
len(overlap_2d1d)

41855